# Recommenders Evaluation

Previously we have seen the basics of Recommender Systems and we have created a variety of models (Most Popular, Content Recommender, Collaborative Filtering Models...).


## Metrics:

1. **Basic metrics** like RMSE or MAE can measure the deviation in the predictions and are very quick to run. The downside typically is that the model tends to achieve a good fit for the items with the most ratings but items with few ratings don't account for much in therms of their impact on the loss function. As a result predictions for items with few ratings will be off spoiling the results.
2. **Specialized metrics** that focus purely on recommendation systems. In this category we have metrics like **Mean Average Precision (MAP)** and **Normalized Discounted Cumulative Gain (NDCG)**.
3. **Description metrics** include metrics that help to characterize the model. For example *item coverage*, calculating how many items are we recommending from the list of all posible items in our database. Or *coverage by year* to measure if we recommend only articles from this year or we are distributing the results across a wider range. Other possibilities that may be relevant depending on the sector include by genre, by author, by manufacturer, by size, by color, etc...

In many recommendation metrics we use <metric@k>, for example MAP@k or NDCG@k.<br>
This means that for every user we ask to get only **k** recommendations and it is based on the accuracy of those k recommendations in which we want to characterize the model and improve the model. The logic behind is that most of the time we only care about the first 5 or 10 recommendations for a user as he is unlikely to explore all the options available, or our interface cannot provide more recommendations in a reasonable way.<br>
So @k is a cutoff that we apply to make sure that our models focus on improving the recommendations that really matter, the ones that the user is going to be exposed to.

### **Mean Average Precision**
MAP is just an average of APs (average precision) for all users in the evaluation. So if we have 1k users, we sum APs for each user and divide the sum by 1k. As mentioned, we will use MAP@k

Important factors to remember:
* We want to recommend at most k items for each user.
* It is worth to submit all k recommendations because we are not penalized for bad guesses.
* Order matters, so it’s better to submit more certain recommendations first followed by recommendations in which we have less confidence.

<br><br>
To understand MAP we do a quick recap of bynary classification metrics Precision and Recall:
\begin{equation*}
Precision:​​​​​​P​​=​​{\frac{\#​correct​positives}{\#​predicted​positives}}
\end{equation*}

\begin{equation*}
Recall:​​​​​​R​​=​​{\frac{\#​correct​positives}{\#​with​condition}}
\end{equation*}

Also in different terminology:<br>
* Precision = (1 - false positive rate)
* Recall = (1 - false negative rate)

These metrics translated to the specific application in Recommender Systems is:
\begin{equation*}
Recommender​​Precision:​​​​​​P​​=​​{\frac{\#​of​our​relevant​recommendations}{\#​of​items​that​we​recommend}}
\end{equation*}

\begin{equation*}
Recommender​​Recall:​​​​​​R​​=​​{\frac{\#​of​our​relevant​recommendations}{\#​of​all​possible​relevant​recommendations}}
\end{equation*}

And to give an example, we are asked to recommend 5 items (k=5) but from the items we have only 3 relevant (m=3) to the user and my succesful predictions are ranked as [0, 1, 1, 0, 0]:<br>
* \# of items that we recommend is 5
* \# of our relevant recommendations is 2
* \# of all possible relevant recommendations is 3
* **precision** is 2/5
* **recall** is 2/3

Now we apply the cutoff at k and the resulting metric that we obtain is:
\begin{equation*}
AP@k​​=​​{\frac{1}{m}}​\sum_{k=1}^N​\left(P(k)​if​k^{th}​item​was​relevant \right)​​=​​{\frac{1}{m}}​\sum_{k=1}^N​P(k)​\cdot​rel(k)
\end{equation*}

With:
\begin{equation*}
rel(k)​​=​​1​​​​--->​​is​relevant
\end{equation*}
\begin{equation*}
rel(k)​​=​​0​​​​--->​not​relevant
\end{equation*}

#### AP example

Given N=3 recommendations (AP@3) for a user, depending on how relevant are the recommendations we can have:

| Recommendations | Precision@k | AP@k |
|:-----------:|:-----------:|:-----------:|
|  [0,0,1]  |   [0,0,1/3]   | (1/3)[1/3] = 0.11 |
|  [0,1,1]  |  [0,1/2,2/3]  | (1/3)[(1/2) + (2/3)] = 0.38 |
|  [1,1,1]  | [1/1,2/2,3/3] | (1/3)[(1) + (2/2) + (3/3)] = 1 |

So the more accurate recommendations I get the larger AP gets because we only sum the k<sup>th</sup> subset precision if the k<sup>th</sup> recommendation was correct so there is a heavy penalty when having incorrect recommendations as well as for not having the correct recommendations at the front.

AP does not penalize having more recommendations but it is really worth to make sure you have the correct ones at the front.

#### MAP@k
Now accounting for all the possible users |U|, we create the averate of the metric for each possible user:
\begin{equation*}
MAP@k​​=​​{\frac{1}{|U|}}​\sum_{u=1}^z​\left( AP@k \right)_u​​=​​{\frac{1}{|U|}}​\sum_{u=1}^z​{\frac{1}{m}}​\sum_{k=1}^N​P_u(k)​\cdot​rel_u(k)
\end{equation*}

Some variations of this metric may try to solve specific applications:
* Avoid penalization when there are more valid recommendations than the ones requested
\begin{equation*}
AP@k​​=​​{\frac{1}{min(m,N)}}​\sum_{k=1}^N​P(k)​\cdot​rel(k)
\end{equation*}

* Define AP = 0 if m = 0 (it will move the final MAP down).

* Expressed in terms of precision and recall
\begin{equation*}
AP@k​​​=​​​\sum_{k=1}^N​(precision​at​k)​\cdot​(change​in​recall​at​k)​​​=​​​\sum_{k=1}^N​P(k)​\Delta r(k)
\end{equation*}
In this formulation we don't need the 1/m as the change in recall is exactly 1/m


#### Notes
- MAP is popular in information retrieval (google search) and recommendations (products).
- Evaluates if the results are relevant and how they are sorted.
- Implies a ranking factor and is very relevant if we want the top recommendations first.


### **Normalized Discounted Cumulative Gain**
NDCG computes a *relevance* score (gain) for each item. If we don't have feedback for an item, the gain is zero. Adding all the gains is the cumulative part but as we want the most relevant items first, we divide each gain by a growing number (typically a logarithm of the item position) which is the *discounting*.

\begin{equation*}
DCG@k​​​=​​​\sum_{i=1}^k​{\frac{rel_i}{log_2(i​+​1)}}
\end{equation*}

Finally because DCGs are not directly comparable across users, we normalize them. The worst possible DCG is zero. For the best, we arrange the items in the test set in the ideal order, take the first *k* items and compute the DCG. Then we divide the raw DCG by this ideal DCT and that's the NDCG@k (a metric between 0 and 1).

\begin{equation*}
NDCG@k​​​=​​​{\frac{DCG@k}{IDCG@k}}
\end{equation*}

An important note is that the test set consists of all items outside the train set including those not ranked by the user. Some people restrict the test set to the user’s held-out ratings, so the recommender’s task is reduced to ordering those relatively few items but this is not a realistic scenario.

#### DCG example:
- Given 6 scores from 0 to 3: [3,2,3,0,1,2]
- CG<sub>6</sub> = 3 + 2 + 3 + 0 + 1 + 2 = 11  Note that changing the order of the document do not change CG.
- DCG adds the relevance of how they are sorted:

| i | rel i | log2(i+1) | (rel i) / (log2 (i+1) |
|:-----------:|:-----------:|:-----------:|:-----------:|
| 1 | 3 |   1   |   3   |
| 2 | 2 | 1.585 | 1.262 |
| 3 | 3 |   2   |  1.5  |
| 4 | 0 | 2.322 |   0   |
| 5 | 1 | 2.585 | 0.387 |
| 6 | 2 | 2.807 | 0.712 |

- DCG<sub>6</sub> = 3 + 1.262 + 1.5 + 0 + 0.387 + 0.712 = 6.861

### MAP vs NDCG
- MAP is a metric for binary feedback while NDCG can be used in any scenario with a relevance score (binary, integer or real).

### Weak and Strong generalization
Users and items can be divided in two groups: those in the training set and those not in it.<br>
The validation of scores for the elements within the training set is the **weak generalization**, and for the others, outside the training set, is the **strong generalization**.

Strong generalization account for real life problems like **cold start**, that is users/items from which we do not have previous information.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,pandas,skmultilearn -g

import os
import sys
import re
import yaml
import dateutil
import watermark
from tqdm import tqdm
from math import floor
from pprint import pprint as pp
from ast import literal_eval

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from pandas.plotting import register_matplotlib_converters    # for pandas_profiling

from fastai.collab import * 

register_matplotlib_converters()                              # for pandas_profiling
sys.path.append(os.pardir)

CPython 3.7.3
IPython 7.5.0

numpy 1.16.4
pandas 0.24.2
skmultilearn unknown

compiler   : GCC 7.3.0
system     : Linux
release    : 5.0.0-27-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : 49826ef1b96601910a3243beb70b988c0ea44380


### Load ratings and metadata

In [2]:
metadata = pd.read_csv('./data/raw/movies_metadata.csv', low_memory=False)
ratings_sm = pd.read_csv("./data/raw/ratings_small.csv", low_memory=False)
credits = pd.read_csv('./data/raw/credits.csv')
keywords = pd.read_csv('./data/raw/keywords.csv')

In [3]:
# Remove rows with bad IDs.
bad_ids =  [idx for idx, k in zip(metadata.index, metadata['id'].values) if not k.isdigit()]
metadata = metadata.drop(bad_ids)

metadata['id'] = metadata['id'].astype('int')
valid_ids = ratings_sm.movieId.unique()
metadata = metadata[metadata['id'].isin(valid_ids)]

metadata.release_date.fillna('1980-01-01', inplace=True)
metadata['release_date'] = metadata.release_date.apply(dateutil.parser.parse)
metadata['overview'] = metadata['overview'].fillna('')

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [4]:
def get_director(x):
    """Get the director's name from the crew feature. If director is not listed, return NaN"""
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [5]:
from src.recommenders import *
from sklearn.model_selection import train_test_split

In [6]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_topn)

In [7]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_str)

In [8]:
def create_combination(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + \
           ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [9]:
metadata['combination'] = metadata.apply(create_combination, axis=1)

#### Split data

In [10]:
X_tr, X_te = train_test_split(ratings_sm, test_size=0.25, random_state=42)
metadata_tr = metadata[metadata.id.isin(X_tr.movieId.unique())]

In [11]:
content_rec = ContentRecommender()
content_rec.train(metadata_tr, 'id', 'overview')

In [12]:
# X_tr.to_csv('./data/processed/X_tr.zip', header=False, compression='zip')
# X_te.to_csv('./data/processed/X_te.zip', header=False, compression='zip')
# metadata.to_csv('./data/processed/metadata.zip', header=False, compression='zip')
# metadata_tr.to_csv('./data/processed/metadata_tr.zip', header=False, compression='zip')

In [13]:
# X_tr = pd.read_csv('./data/processed/X_tr.zip')
# X_te = pd.read_csv('./data/processed/X_te.zip')
# metadata = pd.read_csv('./data/processed/metadata.zip'')
# metadata_tr = pd.read_csv('./data/processed/metadata_tr.zip')

In [15]:
# content_recommendations = content_rec.recommend_df(X_tr, 'userId', 'movieId')

In [ ]:
# Get all the classifiers that we want to evaluate into the list of models...
models = [
    binrel_clf,
    multinomial_csf,
    random_forest_csf,
    extra_tree_csf,
    ensamble_trees_csf,
    multiclass_svm_csf,
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))      # Empty DataFrame to allocate scores
entries = []

# Score all models
for model in models:
    # Extract the model name from the pipeline classifier
    model_name = [(step[1].classifier.__class__.__name__ if hasattr(step[1], 'classifier') 
                   else step[1].__class__.__name__ ) for step in model.steps if step[0] == 'clf'][0]
    accuracies = cross_val_score(model, X_train, df_bin_labels, scoring='accuracy', cv=CV)
    
    for idx, accuracy in enumerate(accuracies):
        entries.append((model_name, idx, accuracy))
        
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_index', 'accuracy'])

In [ ]:
g = sns.boxplot(x='model_name', y='accuracy', data=cv_df)
g.set_xticklabels(g.get_xticklabels(), rotation=30)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()